In [11]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import time

In [12]:
counter = 1
url = 'https://old.reddit.com/r/AmItheAsshole/top/?sort=top&t=all'
headers = {'User-Agent': 'Mozilla/5.0'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')
while counter <= 2500:
    for post in soup.find_all('div', attrs={'class': 'thing', 'data-domain': 'self.AmItheAsshole'}):
        try:
            verdict = post.find("span", attrs={"class": "linkflairlabel"}).text
        except AttributeError:
            verdict = 'No-Flair'
        comment_button = soup.find("a", attrs={"data-event-action":"title"})
        for link in post.find_all('a', class_="comments", text=True):            
            comment_link = link['href']
            comment_page = requests.get(comment_link, headers=headers)
            comment_soup = BeautifulSoup(comment_page.text, 'html.parser')
            md_container = comment_soup.find_all("div", attrs={"class":"usertext-body may-blank-within md-container"})
            for content in md_container[1]:
                try:
                    story = content.text
                except AttributeError:
                    continue                    
            user_comments = {}
            stopper = 11
            if len(md_container) < stopper:
                stopper = len(md_container)
            for i in range(2, stopper):
                for content in md_container[i]:
                    try:
                        comment = content.text
                        user_comments["comment.{}".format(i)] = comment
                    except AttributeError:
                        continue                    
        for span in post:
            post.span.extract()
        title = post.find('p', class_="title").text
        try:
            author = post.find('a', class_='author').text
        except AttributeError:     
            author = 'DELETED'
        comments = post.find('a', class_='comments').text.split()[0]
        likes = post.find("div", attrs={"class": "score likes"}).text
        datetime = post.find("time")
        timestamp = datetime.get('datetime')
        if '-' in author[:1]:
            author.strip('-')
        post_line = [title, author, likes, comments, story, timestamp, verdict]
            
        for keys, values in user_comments.items():
            user_comment = values
            post_line.append(user_comment)
                    
        with open('reddit_aita.csv', 'a',  newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(post_line)
        print(counter)
        counter += 1
    next_button = soup.find("span", class_="next-button")
    next_page_link = next_button.find("a").attrs['href']
    time.sleep(2)
    page = requests.get(next_page_link, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


AttributeError: 'NoneType' object has no attribute 'find'

In [21]:
names=['title', 'author', 'likes', 'comments', 'story', 'timestamp', 'verdict']
for i in range(1, 11):
    comment_number = "user_comment{}".format(i)
    names.append(comment_number)
reddit_aita = pd.read_csv('data/reddit_aita.csv', names=names)

In [19]:
reddit_aita

,title,author,likes,comments,story,timestamp,verdict,user_comment1,user_comment2,user_comment3,user_comment4,user_comment5,user_comment6,user_comment7,user_comment8,user_comment9,user_comment10
0,"UPDATE, AITA for despising my mentally handica...",Renegadesrule33,72.5k,2026,"I'm back like I said I would be,. My original ...",2019-03-11T16:41:02+00:00,UPDATE,"I'm back like I said I would be,. My original ...",Wow. OP's grandfather is a saint.\n,"Too right my guy, the sad thing is is that he ...",Reminds me of my wife's parents in a way. She...,This happened to an ex of mine. Her step fathe...,"That's a good story, always nice to hear a sto...","wait, you adopted your ex?\n","His parents did, while they were together in o...","Yes this, i was 16 at the time she was 15. Unl...",She's not like family. She's your sister. I'm ...
1,UPDATE: AITA for wanting to go to the funeral ...,sgdneyrother,65.0k,2,I want to sincerely thank everyone who comment...,2019-07-29T17:58:12+00:00,Update,I want to sincerely thank everyone who comment...,We've chosen to lock this update due to the hi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,META: Too many AITA commenters advocate too qu...,Cosmohumanist,53.2k,1649,I’ve become frustrated with how quick a lot of...,2019-03-08T16:38:07+00:00,META,I’ve become frustrated with how quick a lot of...,Sometimes I'm surprised by how quickly people ...,Totally agree. And I absolutely appreciate the...,I think that it's only because the most logica...,Leaving a truly bad situation is necessary and...,\nMy issue is when we equate difficult with ba...,I think a huge part of the problem is that if ...,The biggest lesson about love I’ve learned the...,"When people come to r/relationship_advice, the...",Thanks for taking that time for your reply.\nI...
3,AITA for telling my extended family how many m...,Thunderbear998,49.5k,5386,We had a family dinner this evening. My family...,2019-07-27T13:11:51+00:00,Everyone Sucks,We had a family dinner this evening. My family...,I think everything that needs to be said on ei...,ESH One person making personal attacks doesn’t...,"Yeah ESH seems right, not cool of her to be go...",I’ll never understand why people with bigger s...,It's possible that they don't think people wil...,"Some kind of ""too big to fail""? Their skeleton...",That’s a decent point that I hadn’t considered...,Bullies tend to think they are invulnerable ti...,"not necessarily the better person - just a ""co..."
4,AITA for suggesting my 7yo name his new stuffe...,RememberU2U,47.6k,4338,Pretty much the title. My son won a stuffed ti...,2019-08-08T14:32:21+00:00,No A-holes here,Pretty much the title. My son won a stuffed ti...,Locking down this post for all the usual reaso...,NAH\nInappropriate? Yes.\nStill funny? Also ye...,Finally someone who acts like a real person.\n,Yeah this thread is ridiculous. I thought it ...,People who are obsessively dramatic over hypot...,"Specifically, almost fucking all of reddit. Ju...",This sub is worse for this sort of behavior th...,"The format seems to draw those types, especial...",This fuckin sub is becoming ridiculous now\nIt...
5,AITA for despising my mentally handicap sister?,Renegadesrule33,46.8k,2768,The title makes me sound horrible but hear me ...,2019-03-03T20:59:42+00:00,Not the A-hole,The title makes me sound horrible but hear me ...,If you want your comment to count toward judgm...,NTA. These are perfectly legitimate and unders...,"Hey, autistic guy here, can confirm that OP is...",Thank you for sharing and being so open! It so...,No problem at all! And i can kinda relate to h...,Hi! Former genetic engineering major here! Swi...,But you can cure autism by just not getting va...,LMAO I’m gonna be loving it if the “cure” is a...,Given what the commented said further up I can...
6,AITA for giving my ex wife a large amount of m...,binkies123,44.9k,5520,I recently won a ‘fuck you’ amount of money. I...,2019-03-22T11:13:24+00:00,Not the A-hole,I recently won a ‘fuck you’ amount o

In [20]:
reddit_aita.to_csv('data/reddit_aitaR.csv')